# EGamma short exercise

Photons and electrons play a crucial role in the success of CMS. They were vital in the discovery of the Higgs boson and play an important role in searches for new physics and standard model measurements. The CMS detector features a high-resolution electromagnetic calorimeter (ECAL) and the world's largest active silicon tracker. These detectors provide clean object identification and excellent energy and momentum resolution of photons and electrons. In this exercise, we will examine photon and electron identification and study how these objects are used in physics analyses. We will start with [NanoAOD](https://twiki.cern.ch/twiki/bin/view/CMSPublic/WorkBookNanoAOD) samples.

Table of contents:
   * [Finding samples](#Finding-samples)
   * [NanoAOD](#NanoAOD)
   * [Discriminating variables](#Discriminating-variables)

## Finding samples
Let's find two simulation samples to use for the remainder of the exercise: a $Z\rightarrow \ell^{+}\ell^{-}$ sample, which will contain two electrons in slightly more than 1/3 of the events (**Q: why greater than a third?**); and a sample of $\gamma+\text{jets}$ events, where a photon is radiated off either initial- or final-state quarks.  To find samples via a web browser, one can go to the [DAS client](https://cmsweb.cern.ch/das/) page.  Here, we use the `dasgoclient` command-line utility, using Jupyter notebook's shell execution shortcut.

The query format is `dasgoclient --query="key requirement [requirement ...]"` where the key is what type of record we are looking for, and each requirement helps filter results.  Here we want to find some datasets that match a wildcard pattern.  Datasets in CMS (mostly) follow a structured naming convention,

```/Primary/Campaign-Conditions-Revision/Datatier```,

where: `Primary` is the primary dataset type, either a particular set of triggers for real data, or a particular physics process for simulation; `Campaign` encodes the data reprocessing or simulation generating campaign, used by CMS computing to organize the computing workflows; `Conditions` specifies in more detail the reconstruction and/or simulation conditions of the dataset; `Revision` allows for replacement of invalid datasets or extensions (e.g. `_ext1` you might sometimes see); and `Datatier` describes what kind of format and content is in the ROOT files inside the dataset.  Below, we search for a simulation dataset for the Drell-Yan ($Z/\gamma^{*}$) process, where the decay is to two leptons (`ToLL`) and a generator-level mass cut of 50 GeV is imposed.  Adding `_Tune*` helps reduce the number of matches where additional generator cuts are present.  Try removing it to see what kind of exclusive phase spases samples are commonly generated for.  For the generation campaign, we choose the Autumn18 production, which corresponds to the 2018 data-taking conditions.

In [1]:
!dasgoclient --query="dataset dataset=/DYJetsToLL_M-50_Tune*/RunIIAutumn18NanoAODv5*/NANOAODSIM"

/DYJetsToLL_M-50_TuneCP5_13TeV-amcatnloFXFX-pythia8/RunIIAutumn18NanoAODv5-Nano1June2019_102X_upgrade2018_realistic_v19-v1/NANOAODSIM
/DYJetsToLL_M-50_TuneCP5_13TeV-amcatnloFXFX-pythia8/RunIIAutumn18NanoAODv5-Nano1June2019_102X_upgrade2018_realistic_v19_ext2-v1/NANOAODSIM
/DYJetsToLL_M-50_TuneCP5_13TeV-madgraphMLM-pythia8/RunIIAutumn18NanoAODv5-Nano1June2019_102X_upgrade2018_realistic_v19-v1/NANOAODSIM


We see three results, all are produced with the [MadGraph5_aMC@NLO](https://twiki.cern.ch/twiki/bin/view/CMS/CitationsForGenerators#MadGraph5_a_MCatNLO) generator, where one simulates the Z+jets process at leading order (`madgraphMLM`), one at NLO (`amcatnloFXFX`), and one which is just an extension to increase the number of simulated events.  From here, it is possible albeit convoluted to see the original run cards that configured the MadGraph generator.  We tracked them down for you, so you can see what actually configures the event generation: [madgraphMLM card](https://github.com/cms-sw/genproductions/tree/1e06dc8f0c86ab6fdb547b27c43c92b022608b80/bin/MadGraph5_aMCatNLO/cards/production/2017/13TeV/DYJets_HT_LO_MLM/DYJets_HT_mll50/DYJets_HT-incl), [amcatnloFXFX card](https://github.com/cms-sw/genproductions/tree/59237cf5f41aed600b63fa8d071441129aeb6f9a/bin/MadGraph5_aMCatNLO/cards/production/2017/13TeV/dyellell012j_5f_NLO_FXFX
).

Let's use the LO sample.  Below, we query the files belonging to the dataset, and save them to a python variable.

In [2]:
dy_files = !dasgoclient --query="file dataset=/DYJetsToLL_M-50_TuneCP5_13TeV-madgraphMLM-pythia8/RunIIAutumn18NanoAODv5-Nano1June2019_102X_upgrade2018_realistic_v19-v1/NANOAODSIM"
print("Number of files:", len(dy_files))

Number of files: 53


Now, for the photon event samples, since the inclusive production phase space is so large, we have to impose a minimum transverse energy sum requirement (`HT`, e.g. $\sum_{\text{particles}} p_{T}$) to be able to generate enough events in the phase space we are interested in (which is dictated by photon trigger thresholds).

In [3]:
!dasgoclient --query="dataset dataset=/GJets_HT-*/RunIIAutumn18NanoAODv5*/NANOAODSIM"

/GJets_HT-100To200_DoubleEMEnriched_MGG-80toInf_TuneCP5_13TeV-madgraphMLM-pythia8/RunIIAutumn18NanoAODv5-Nano1June2019_102X_upgrade2018_realistic_v19-v1/NANOAODSIM
/GJets_HT-100To200_TuneCP5_13TeV-madgraphMLM-pythia8/RunIIAutumn18NanoAODv5-Nano1June2019_4cores5k_102X_upgrade2018_realistic_v19-v1/NANOAODSIM
/GJets_HT-200To400_DoubleEMEnriched_MGG-80toInf_TuneCP5_13TeV-madgraphMLM-pythia8/RunIIAutumn18NanoAODv5-Nano1June2019_102X_upgrade2018_realistic_v19-v1/NANOAODSIM
/GJets_HT-200To400_TuneCP5_13TeV-madgraphMLM-pythia8/RunIIAutumn18NanoAODv5-Nano1June2019_102X_upgrade2018_realistic_v19-v1/NANOAODSIM
/GJets_HT-400To600_DoubleEMEnriched_MGG-80toInf_TuneCP5_13TeV-madgraphMLM-pythia8/RunIIAutumn18NanoAODv5-Nano1June2019_102X_upgrade2018_realistic_v19-v1/NANOAODSIM
/GJets_HT-400To600_TuneCP5_13TeV-madgraphMLM-pythia8/RunIIAutumn18NanoAODv5-Nano1June2019_102X_upgrade2018_realistic_v19-v1/NANOAODSIM
/GJets_HT-40To100_DoubleEMEnriched_MGG-80toInf_TuneCP5_13TeV-madgraphMLM-pythia8/RunIIAutumn18

We see two types of generation configurations, one is inclusive and the other enforces at least two generator-level photons with a minimum invariant mass cut between them: `DoubleEMEnriched_MGG-80toInf`. (**Q: what might these `DoubleEMEnriched_MGG-80toInf` samples be useful for?**)

Let's use just one HT bin for our photon sample.  We will investigate how much that restricts the generated photon $p_{T}$.

In [4]:
gjets_files = !dasgoclient --query="file dataset=/GJets_HT-200To400_TuneCP5_13TeV-madgraphMLM-pythia8/RunIIAutumn18NanoAODv5-Nano1June2019_102X_upgrade2018_realistic_v19-v1/NANOAODSIM"
print("Number of files:", len(gjets_files))

Number of files: 17


The files we've identified above are all LFNs (Logical File Names), and they alone don't specify where and how to read them.  For that, we need to specify a PFN (Physical File Name).  CMS operates a distributed data access system known as [AAA](https://twiki.cern.ch/twiki/bin/view/CMSPublic/WorkBookXrootdService) which makes files available via the [xrootd protocol](https://xrootd.slac.stanford.edu/) regardless of their storage location.  In short, adding the prefix `root://cmsxrootd.fnal.gov/` to all the files makes them readable.

In [5]:
redirector = 'root://cmsxrootd.fnal.gov/'
dy_files = [redirector + f for f in dy_files]
gjets_files = [redirector + f for f in gjets_files]

## NanoAOD
In previous iterations of this exercise, students started by running an _NTuplizer_ over MiniAOD samples.  The [MiniAOD datatier](https://twiki.cern.ch/twiki/bin/view/CMSPublic/WorkBookMiniAOD) contains enough detailed information to cover essentially all typical CMS analyses.  The EGamma-related content is discussed further in the [EgammaMiniAODV2](https://twiki.cern.ch/twiki/bin/viewauth/CMS/EgammaMiniAODV2) TWiki.  A concise example of accessing MiniAOD photon content is given [here](https://twiki.cern.ch/twiki/bin/view/CMSPublic/WorkBookMiniAOD2017#Example_code_accessing_all_high).

A plethora of nTuplizers have been written over the past few years, all essentially extracting similar information from MiniAOD.  In an attempt to reduce code maintenence burden, a common nTuple, [NanoAOD](https://twiki.cern.ch/twiki/bin/view/CMSPublic/WorkBookNanoAOD) was created.  It contains less information than MiniAOD, and is a much more compact file format, averaging 1-2 kB per event vs. MiniAOD's 30-50 kB per event.  In addition, NanoAOD saves all information in basic types (like `int`, `float`, etc.) rather than the [EDM format](https://twiki.cern.ch/twiki/bin/view/CMSPublic/WorkBookCMSSWFramework#EdM) used for all parent datatiers.  This means that no CMSSW environment is necessary to read these files.  In fact, we will not even use the ROOT framework to analyze these files in this exercise!

For analyses that require information not available in standard NanoAOD, one can create _private NanoAOD_, where  missing variables and/or CMSSW framework modules are added to the [configuration](https://github.com/cms-sw/cmssw/blob/master/PhysicsTools/NanoAOD/python/nano_cff.py) and then CRAB workflows are used to re-extract the information from the parent MiniAOD dataset.  The CMS [cross-POG forum](https://indico.cern.ch/category/5517/) exists to also allow users to petition for certain variables to be added to central NanoAOD.  For the purposes of this exercise, all standard EGamma variables can already be found in NanoAOD.

Below, we examine the content of a NanoAOD file using [uproot](https://uproot.readthedocs.io/en/latest/?badge=latest#).

In [6]:
import uproot
file = uproot.open(dy_files[0])
{k: file[k] for k in file}

{b'tag;1': b'untagged',
 b'Events;1': <TTree b'Events' at 0x7fa43fe0fd68>,
 b'LuminosityBlocks;1': <TTree b'LuminosityBlocks' at 0x7fa434112860>,
 b'Runs;1': <TTree b'Runs' at 0x7fa434112ba8>,
 b'MetaData;1': <TTree b'MetaData' at 0x7fa4341225f8>,
 b'ParameterSets;1': <TTree b'ParameterSets' at 0x7fa4341227b8>}

The item of interest here is the `Events` [TTree](https://root.cern.ch/doc/master/classTTree.html) object.  Let's see what is inside it:

In [7]:
tree = file['Events']
tree.show()

run                        (no streamer)              asdtype('>u4')
luminosityBlock            (no streamer)              asdtype('>u4')
event                      (no streamer)              asdtype('>u8')
HTXS_Higgs_pt              (no streamer)              asdtype('>f4')
HTXS_Higgs_y               (no streamer)              asdtype('>f4')
HTXS_stage1_1_cat_pTjet25GeV
                           (no streamer)              asdtype('>i4')
HTXS_stage1_1_cat_pTjet30GeV
                           (no streamer)              asdtype('>i4')
HTXS_stage1_1_fine_cat_pTjet25GeV
                           (no streamer)              asdtype('>i4')
HTXS_stage1_1_fine_cat_pTjet30GeV
                           (no streamer)              asdtype('>i4')
HTXS_stage_0               (no streamer)              asdtype('>i4')
HTXS_stage_1_pTjet25       (no streamer)              asdtype('>i4')
HTXS_stage_1_pTjet30       (no streamer)              asdtype('>i4')
HTXS_njets25               (no streamer)      

Wow, that's a lot of stuff!  What is shown here is each column (a ROOT [TBranch](https://root.cern.ch/doc/master/classTBranch.html)) inside the tree, along with some interpretation information.  Another source of this information is [this website](https://cms-nanoaod-integration.web.cern.ch/integration/master-102X/mc102X_doc.html) (and its' parent folder contains branch lists for other NanoAOD versions.)

To help organize all this content, we'll use the [coffea](https://coffeateam.github.io/coffea/) NanoEvents class:

In [12]:
from coffea.nanoaod import NanoEvents

dyevents = NanoEvents.from_file(dy_files[0])

In [16]:
dyevents.columns

['Pileup',
 'MET',
 'SoftActivityJetHT2',
 'PV',
 'LHEScaleWeight',
 'TkMET',
 'event',
 'SoftActivityJetHT5',
 'fixedGridRhoFastjetCentralChargedPileUp',
 'SoftActivityJet',
 'L1Reco',
 'SoftActivityJetHT',
 'LHEReweightingWeight',
 'Electron',
 'PSWeight',
 'SoftActivityJetHT10',
 'FatJet',
 'LHEPdfWeight',
 'ChsMET',
 'genTtbarId',
 'CorrT1METJet',
 'GenMET',
 'run',
 'fixedGridRhoFastjetCentralNeutral',
 'Muon',
 'Generator',
 'LHEPart',
 'SubGenJetAK8',
 'Flag',
 'fixedGridRhoFastjetCentral',
 'GenJetAK8',
 'SubJet',
 'fixedGridRhoFastjetAll',
 'GenDressedLepton',
 'L1',
 'IsoTrack',
 'OtherPV',
 'genWeight',
 'fixedGridRhoFastjetCentralCalo',
 'LHE',
 'GenJet',
 'SoftActivityJetNjets10',
 'TrigObj',
 'luminosityBlock',
 'Tau',
 'GenPart',
 'GenVisTau',
 'L1simulation',
 'btagWeight',
 'Jet',
 'Photon',
 'CaloMET',
 'SV',
 'SoftActivityJetNjets5',
 'PuppiMET',
 'LHEWeight',
 'HLTriggerFinalPath',
 'HTXS',
 'HLT',
 'RawMET',
 'SoftActivityJetNjets2',
 'HLTriggerFirstPath']

Ok, slightly more manageable content. Try replacing the above cell with `dyevents.Electron.columns` and `dyevents.Photon.columns` to see what EGamma-related columns exist in the tree.

## Discriminating variables
   * Shower Shape Variable, sigmaIetaIeta: The energy weighted standard deviation of single crystal eta within the 5 × 5 crystals centered at the crystal with maximum energy.
    <img src="SWGuideCMSDataAnalysisSchoolLPC2019egamma/SigmaIEtaIEta_definition.png" alt="SigmaIEtaIEta_definition.png" width='400'  /> 
where the sum runs over the 5 × 5 crystal matrix around the most energetic crystal in the SuperCluster, and the eta distances are measured in units of the crystal size in the eta direction. This variable represents the second moment of the energy distribution along the eta coordinate.

   * R9: Ratio of the energy deposited in ECAL inside a 3 × 3 crystal matrix centered on the most energetic crystal in the supercluster and the superCluster raw energy.
   * H/E: Ratio of the energy deposited in the single closest HCAL tower to the supercluster position inside a cone of radius 0.15 in the eta - phi plane centered on the photon direction and the energy deposited in ECAL to that supercluster.
   * PF Photon Isolation (IPh): Sum of Et of all PF photons which are not in the “footprint” of the candidate photon and are within a cone of DeltaR = 0.3.
   * PF Charged Hadron Isolation (ICh ): Sum of Pt of all PF charged hadrons which are associated to the primary vertex, not in the “footprint” of the candidate photon, and within a cone of DeltaR = 0.3.
   * PF Neutral Hadron Isolation (INh ): Sum of Et of all PF neutral hadrons which are not in the “footprint” of the candidate photon and are within a cone of DeltaR = 0.3.
   * Conversion safe electron veto : This cut replaces the detector based pixel seed used to distinguish between photon and electron. It checks the DeltaR distance between the photon direction and the nearest electron track and remove photons if there is a GsfElectron matching to the photon supercluster, with no missing hits and having no matching reconstructed conversion.
   * Electron mini isolation: Uses a cone whose size varies as a function of the electron pT in order to reduce accidental overlap from hard scatter or pileup processes.

Note: The photon isolation cone is also susceptible to pileup from interactions not corresponding to the primary vertex. The correction from the pile-up energy contribution to the photon identification is applied using the following equation: Iso (new) = Iso (original) - (Rho_event *Effective Area), where Rho_event is the energy density computed using FastJet package. Rho_event is the median background density per unit area and a measure of the pile up activity in the event. The effective area is defined as the ratio of the slope obtained from linearly fitting Iso(Nvtx ) to the one from linearly Jets fitting Rho_event(Nvtx). The values of effective area for all three isolation criteria (photon, neutral and charged) are provided by the EGamma POG separately for barrel and endcap regions.
